In [ ]:
import random
import csv
import time
import os
import socket
import numpy as np
import inspyred
from inspyred import ec
from inspyred.ec import terminators
from codecarbon import EmissionsTracker

# Configuración de experimento
RUNS_PER_CONFIG = 10  # Ajustar si se desean múltiples corridas por configuración
POPULATION_SIZES = [2**6, 2**10, 2**14]
CROSSOVER_RATES = [0.2, 0.01, 0.8]
MUTATION_RATE = 0.1
INDIVIDUAL_SIZE = 1024
MAX_TIME_SECONDS = 120  # segundos
MAX_GENERATIONS = 100000

# Obtener nombre del ordenador
COMPUTER_NAME = socket.gethostname()
output_file = f'resultados_inspyred_{COMPUTER_NAME}_sphere.csv'
codecarbon_file = f'codecarbon_inspyred_{COMPUTER_NAME}_sphere.csv'

# Dominio y escala para Sphere
LOW, UP = -5.12, 5.12
F_MAX = INDIVIDUAL_SIZE * (UP ** 2)  # Peor caso para normalización

# Verificar si el archivo de resultados existe
file_exists = os.path.isfile(output_file)

# Clase para almacenar las estadísticas de evolución
class EvolutionStats:
    def __init__(self):
        self.initial_fitness = 0
        self.best_fitness = 0
        self.best_raw_fitness = 0
        self.gen_fitness_max = 0
        self.current_generation = 0
        self.termination_cause = "timeout"

# Instancia global para almacenar estadísticas
stats = EvolutionStats()

# Función para normalizar el fitness de Sphere a porcentaje (0-1)
def normalizar_fitness(raw_fitness):
    """Normaliza el fitness de Sphere donde 1 es el mejor (0) y 0 es el peor (F_MAX)"""
    if raw_fitness >= F_MAX:
        return 0.0
    return (1.0 - (raw_fitness / F_MAX)) 

# Función evaluadora para Sphere (menor es mejor)
def evaluador_sphere(candidates, args):
    """Evalúa los candidatos usando la función Sphere"""
    fitness_values = [sum(x**2 for x in candidate) for candidate in candidates]
    
    # Convertir a problema de maximización para que sea compatible con inspyred
    return [-fitness for fitness in fitness_values]

# VERSIÓN CORREGIDA: Implementación del operador SBX como variator para Inspyred
def sbx_crossover_variator(random, candidates, args):
    """
    Operador de cruza SBX (Simulated Binary Crossover) implementado como variator.
    Esta función se aplica a toda la población de candidatos.
    """
    crossover_rate = args.setdefault('crossover_rate', 1.0)
    sbx_eta = args.setdefault('sbx_eta', 20.0)  # Usar mismo eta que en DEAP
    bounds = (LOW, UP)
    
    # Crear una nueva lista para los hijos
    offspring = []
    
    # Procesar cada par de padres
    for i in range(0, len(candidates) - 1, 2):
        mom = candidates[i]
        dad = candidates[i + 1]
        
        # Si no se hace cruza
        if random.random() >= crossover_rate:
            offspring.append(mom[:])
            offspring.append(dad[:])
            continue
        
        child1 = mom[:]
        child2 = dad[:]
        
        # Para cada componente
        for j in range(len(mom)):
            if random.random() < 0.5:  # Probabilidad por componente
                if abs(mom[j] - dad[j]) > 1e-10:  # Evitar división por cero
                    if mom[j] > dad[j]:
                        mom_val, dad_val = dad[j], mom[j]
                    else:
                        mom_val, dad_val = mom[j], dad[j]
                    
                    y1, y2 = mom_val, dad_val
                    
                    # Cálculo SBX
                    u = random.random()
                    beta = 1.0 + (2.0 * (y1 - bounds[0]) / (y2 - y1))
                    alpha = 2.0 - beta ** (-(sbx_eta + 1.0))
                    
                    if u <= 1.0 / alpha:
                        beta_q = (u * alpha) ** (1.0 / (sbx_eta + 1.0))
                    else:
                        beta_q = (1.0 / (2.0 - u * alpha)) ** (1.0 / (sbx_eta + 1.0))
                    
                    # Generar hijos
                    c1 = 0.5 * ((y1 + y2) - beta_q * (y2 - y1))
                    c2 = 0.5 * ((y1 + y2) + beta_q * (y2 - y1))
                    
                    # Restringir a límites
                    c1 = max(min(c1, bounds[1]), bounds[0])
                    c2 = max(min(c2, bounds[1]), bounds[0])
                    
                    child1[j] = c1
                    child2[j] = c2
        
        offspring.append(child1)
        offspring.append(child2)
    
    # Si el número de candidatos es impar, añadir el último sin modificar
    if len(candidates) % 2 != 0:
        offspring.append(candidates[-1][:])
    
    return offspring

def polynomial_mutation_variator(random, candidates, args):
    """
    Implementación de mutación polinómica acotada como variator.
    Esta función se aplica a toda la población de candidatos.
    """
    mut_rate = args.setdefault('mutation_rate', 0.1)  # Probabilidad de mutación por gen
    eta = args.setdefault('eta', 20.0)  # Parámetro de distribución 
    bounder = args.setdefault('_ec').bounder
    low, up = LOW, UP  # Usando las constantes globales
    
    mutants = []
    
    for candidate in candidates:
        mutant = candidate[:]
        
        # Para cada gen en el individuo
        for i in range(len(candidate)):
            if random.random() < mut_rate:
                x = mutant[i]
                
                # Implementación de la distribución polinómica
                delta_1 = (x - low) / (up - low)
                delta_2 = (up - x) / (up - low)
                
                rand = random.random()
                mut_pow = 1.0 / (eta + 1.0)
                
                if rand < 0.5:
                    xy = 1.0 - delta_1
                    val = 2.0 * rand + (1.0 - 2.0 * rand) * (xy ** (eta + 1.0))
                    delta_q = val ** mut_pow - 1.0
                else:
                    xy = 1.0 - delta_2
                    val = 2.0 * (1.0 - rand) + 2.0 * (rand - 0.5) * (xy ** (eta + 1.0))
                    delta_q = 1.0 - val ** mut_pow
                
                # Aplicar el cambio
                x = x + delta_q * (up - low)
                
                # Asegurar que queda dentro de los límites
                x = min(max(x, low), up)
                
                mutant[i] = x
        
        mutants.append(mutant)
    
    return mutants

# Función para crear un generador de individuos
def generator(random, args):
    """Genera individuos con valores aleatorios dentro de los límites"""
    size = args.get('individual_size', INDIVIDUAL_SIZE)
    return [random.uniform(LOW, UP) for _ in range(size)]

# Función time_termination personalizada para detener si se excede el tiempo máximo
def time_termination(population, num_generations, num_evaluations, args):
    """Termina si se excede el tiempo máximo especificado"""
    global stats
    
    start_time = args.get('start_time', None)
    max_time = args.get('max_time', MAX_TIME_SECONDS)
    
    # Actualizar generación actual
    stats.current_generation = num_generations
    
    if start_time is None:
        return False
    
    elapsed_time = time.time() - start_time
    
    if elapsed_time >= max_time:
        stats.termination_cause = "timeout"
        return True
    
    return False

# Función para detectar convergencia
def convergence_termination(population, num_generations, num_evaluations, args):
    """Termina si encontramos una solución óptima"""
    global stats
    
    if not population:
        return False
        
    best_fitness = max([ind.fitness for ind in population])
    # Convertir a fitness normalizado
    raw_fitness = -best_fitness  # Convertir de vuelta a problema de minimización
    normalized_fitness = normalizar_fitness(raw_fitness)
    
    # Detectar si hemos alcanzado convergencia (fitness muy cercano a 100%)
    if normalized_fitness >= 1.0:
        stats.termination_cause = "convergence"
        return True
    
    return False

# Observer para registrar el mejor fitness en cada generación
def best_fitness_observer(population, num_generations, num_evaluations, args):
    """Registra el mejor fitness en cada generación"""
    global stats
    
    if not population:
        return
        
    best_ind = max(population)
    raw_fitness = -best_ind.fitness  # Convertir de vuelta a problema de minimización
    normalized_fitness = normalizar_fitness(raw_fitness)
    
    # Si es la primera generación, registrar el fitness inicial
    if num_generations == 0:
        stats.initial_fitness = normalized_fitness
        stats.best_fitness = normalized_fitness
        stats.best_raw_fitness = raw_fitness
    
    # Actualizar mejor fitness global si es necesario
    if normalized_fitness > stats.best_fitness:
        stats.best_fitness = normalized_fitness
        stats.best_raw_fitness = raw_fitness
        stats.gen_fitness_max = num_generations
    
def main():
    # Crear archivo de resultados si no existe
    with open(output_file, 'a', newline='') as csvfile:
        fieldnames = [
            'framework', 'poblacion', 'cruce', 'mutacion',
            'generaciones_executadas', 'fitness_inicial_max(%)',
            'fitness_maximo_alcanzado(%)', 'variacion_fitness(%)',
            'gen_fitness_max', 'motivo_parada', 'tiempo_total_segundos',
            'emisiones_kgCO2eq', 'timestamp'
        ]
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        
        if not file_exists:
            writer.writeheader()
        
        # Probar cada combinación de parámetros
        for pop_size in POPULATION_SIZES:
            for cx_rate in CROSSOVER_RATES:
                for run in range(1, RUNS_PER_CONFIG + 1):
                    print(f"Ejecutando: Población={pop_size}, Cruce={cx_rate}, Ejecución={run}")
                    
                    # Reiniciar las estadísticas para esta ejecución
                    global stats
                    stats = EvolutionStats()
                    
                    # Configurar el seguimiento de emisiones
                    tracker = EmissionsTracker(
                        project_name="Sphere_Experiment",
                        output_dir='.', 
                        output_file=codecarbon_file, 
                        save_to_file=True, 
                        log_level="critical",
                        tracking_mode="process",
                        allow_multiple_runs=False
                    )
                    
                    # Iniciar seguimiento de emisiones
                    tracker.start()
                    start_time = time.time()
                    
                    # Configurar el algoritmo genético
                    rand = random.Random()
                    rand.seed(time.time() + run)  # Semilla aleatoria para cada ejecución
                    
                    ea = ec.GA(rand)
                    ea.selector = inspyred.ec.selectors.tournament_selection
                    ea.variator = [sbx_crossover_variator, polynomial_mutation_variator]
                    ea.replacer = inspyred.ec.replacers.generational_replacement
                    
                    # Configurar los terminadores combinados
                    ea.terminator = [
                        inspyred.ec.terminators.generation_termination,
                        time_termination,
                        convergence_termination
                    ]
                    
                    # Añadir el observador para seguir el mejor fitness
                    ea.observer = best_fitness_observer
                    
                    # Ejecutar el algoritmo genético con una sola llamada a evolve
                    final_pop = ea.evolve(generator=generator,
                                     evaluator=evaluador_sphere,
                                     pop_size=pop_size,
                                     maximize=True,
                                     bounder=inspyred.ec.Bounder(LOW, UP),
                                     max_generations=MAX_GENERATIONS,
                                     num_selected=pop_size,
                                     individual_size=INDIVIDUAL_SIZE,
                                     crossover_rate=cx_rate,
                                     mutation_rate=MUTATION_RATE,
                                     sbx_eta=20.0,
                                     eta=20.0,
                                     start_time=start_time,
                                     max_time=MAX_TIME_SECONDS)
                    
                    # Obtener tiempo transcurrido
                    end_time = time.time()
                    elapsed_time = end_time - start_time
                    
                    # Detener el seguimiento de emisiones
                    emissions = tracker.stop()
                    if emissions is None:
                        emissions = 0
                    
                    # Usar los datos almacenados en nuestro objeto stats
                    initial_fitness = stats.initial_fitness
                    best_fitness = stats.best_fitness
                    gen_fitness_max = stats.gen_fitness_max
                    motivo_parada = stats.termination_cause
                    fitness_variation = best_fitness - initial_fitness
                    generations = stats.current_generation + 1  # +1 porque empezamos desde 0
                    
                    # Guardar resultados
                    writer.writerow({
                        'framework': 'INSPYRED_SBX',
                        'poblacion': pop_size,
                        'cruce': cx_rate,
                        'mutacion': MUTATION_RATE,
                        'generaciones_executadas': generations,
                        'fitness_inicial_max(%)': f'{initial_fitness:.6f}',
                        'fitness_maximo_alcanzado(%)': f'{best_fitness:.6f}',
                        'variacion_fitness(%)': f'{fitness_variation:.6f}',
                        'gen_fitness_max': gen_fitness_max,
                        'motivo_parada': motivo_parada,
                        'tiempo_total_segundos': f'{elapsed_time:.2f}',
                        'emisiones_kgCO2eq': f'{emissions:.6f}',
                        'timestamp': time.strftime('%Y-%m-%d %H:%M:%S')
                    })
                    
                    # Asegurarse de que los datos se escriban al archivo después de cada ejecución
                    csvfile.flush()
                    
                    print(f"Finalizado: Fitness inicial={initial_fitness:.2f}%, Mejor fitness={best_fitness:.2f}%, Variación={fitness_variation:.2f}%")
                    print(f"Generaciones={generations}, Tiempo: {elapsed_time:.2f}s, Emisiones: {emissions:.6f} kgCO2eq")
                    print(f"Motivo de parada: {motivo_parada}")
                    print("-" * 50)

if __name__ == "__main__":
    main()